## Install Ollama

Navigate to https://ollama.com/download and install the appropriate version. You can also use the docker image to run Ollama as a container.

In [2]:
# Run ollama on a separate shell to be able to call it in the rest of the notebook

!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: address already in use


## Use Ollama with Python

Install the python client for `ollama`

In [3]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


Pull the Phi 3 model

In [8]:
import ollama # Import the library

ollama.pull("phi3:3.8b")

{'status': 'success'}

Let's make a request to the LLM. We will use `Phi 3 3.8b`, which we downloaded a few moments ago

In [9]:
# Make a "chat" call and specify the messages
response = ollama.chat(model='phi3:3.8b', messages=[
  {
    'role': 'user',
    'content': 'Who are you?',
  },
])

# Print the contents of the respone
print(response['message']['content'])

I'm Phi, an artificial intelligence developed to interact and provide information across various domains. How can I help you today?


In addition to the contents, each ollama response comes with additional metadata, let's peek into it

In [10]:
response

{'model': 'phi3:3.8b',
 'created_at': '2024-09-10T02:29:31.263125Z',
 'message': {'role': 'assistant',
  'content': "I'm Phi, an artificial intelligence developed to interact and provide information across various domains. How can I help you today?"},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1160719584,
 'load_duration': 791851125,
 'prompt_eval_count': 13,
 'prompt_eval_duration': 78471000,
 'eval_count': 27,
 'eval_duration': 289441000}

Let's build a small chat functionality using Ollama

In [13]:
# Create a type alias for chat history
ChatHistory = list[dict[str, str]]
# We will define a helper function that takes as input a message and the chat history
def chat(message:str, history:ChatHistory = list(), model="llama3.1") -> None:
	# Append the input to the conversation history
	history.append({"role":"user", "content":message})
	# Call the streaming function
	stream = ollama.chat(
		model=model,
		messages=history,
		options={
			"temperature":0., 			# Set the temperature to zero to get consistent outputs
		},		
		stream=True,
	)

	chunks = []
	for chunk in stream:
		# Let's accumulate the chunk
		chunks.append(chunk)
		# Also  print it
		print(chunk['message']['content'], end='', flush=True)
	# Once Ollama finish, let's update the chat history
	message = {'role':chunks[0]['message']['role'], 'content':''.join([chunk['message']['content'] for chunk in chunks])}
	# Add it to the chat history
	history.append(message)

# Test it
chat("Howdy!")

Howdy back atcha! What brings you to these here parts?

# Using Phi 3 as a coding assistant

We are going to let Phi 3 code a project for us. We will start describing the application we want, then iterate over each specific component

In [17]:
history = [{
	"role":"system",
	"content":"You are a coding assistant that helps a software developer"
	}]

chat("We will create together a REST API to mantain a database catalog of books. I will describe the requirements and you will be in charge of the details. The app will be primarily built using python, FAST API, and SQLAlchemy. Your first task is to define the directory structure of the application", history)

Based on best practices for structuring a Python project with FastAPI and SQLAlchemy, I propose the following directory structure:

```bash
book_catalog_api/
├── app/
│   ├── __init__.py
│   ├── main.py  # The entry point of our API
│   ├── models/  # Database models using SQLAlchemy
│   │   ├── __init__.py
│   │   ├── book.py
│   │   └── user.py (if we need to manage users for authentication)
│   ├── routes/
│   │   ├── __init__.py
│   │   ├── books.py  # API endpoints for managing books
│   │   └── users.py (if we implement user management)
│   ├── schemas/  # JSON schema definitions for request and response bodies
│   │   ├── __init__.py
│   │   ├── book_schema.py
│   │   └── user_schema.py (if needed)
│   ├── services/
│   │   ├── __init__.py
│   │   ├── book_service.py  # Business logic for managing books
│   │   └── user_service.py (if we implement user management)
│   └── utils/     # Utility functions and classes
│       ├── __init__.py
│       └── db_utils.py  # Functions for 

We will start with the models

In [18]:
chat("Write down the contents for the file named `models/book.py`")

Here are some example contents for the file `models/book.py`:

**models/book.py**
```python
from .base import BookBaseModel

class Book(BookBaseModel):
    """
    A model representing a book.
    """

    title = models.CharField(max_length=200)
    author = models.CharField(max_length=100)
    publication_date = models.DateField()
    isbn = models.CharField(max_length=13, unique=True)

    def __str__(self):
        return self.title

class BookReview(models.Model):
    """
    A model representing a review for a book.
    """

    book = models.ForeignKey(Book, on_delete=models.CASCADE)
    rating = models.IntegerField()
    review_text = models.TextField()

    def __str__(self):
        return f"{self.book.title} - {self.rating}/5"
```
In this example, we have two models: `Book` and `BookReview`. The `Book` model inherits from a base model (`BookBaseModel`) that might contain common fields or methods for all book-related models.

The `BookReview` model has a foreign key to the `B

In [19]:
chat("Write down the contents for the file named `models/user.py`")

Here are some example contents for the file `models/user.py`:

**models/user.py**
```python
from django.contrib.auth.models import AbstractBaseUser, PermissionsMixin
from .base import UserBaseModel

class User(UserBaseModel, AbstractBaseUser, PermissionsMixin):
    """
    A custom user model.
    """

    username = models.CharField(max_length=150, unique=True)
    email = models.EmailField(unique=True)
    first_name = models.CharField(max_length=30, blank=True)
    last_name = models.CharField(max_length=150, blank=True)

    USERNAME_FIELD = 'username'
    REQUIRED_FIELDS = ['email']

    def __str__(self):
        return self.username

class UserProfile(models.Model):
    """
    A model representing a user's profile.
    """

    user = models.OneToOneField(User, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    avatar = models.ImageField(upload_to='avatars/', blank=True)

    def __str__(self):
        return f"{self.user.username}'s Profile"
```
In this examp

In [20]:
chat("Remember that you're using SQLAlchemy, not Django. Based on this, generate again the contents of `user.py`")

Here are the contents for the file `models/user.py`, using SQLAlchemy:

**models/user.py**
```python
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from .base import Base

class User(Base):
    """
    A model representing a user.
    """

    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String(150), unique=True)
    email = Column(String(100), unique=True)
    first_name = Column(String(30), nullable=True)
    last_name = Column(String(150), nullable=True)

class UserProfile(Base):
    """
    A model representing a user's profile.
    """

    __tablename__ = 'user_profiles'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'), unique=True)
    bio = Column(String(500), nullable=True)
    avatar = Column(String(255), nullable=True)

class UserSession(Base):
    """
    A model representing a user's session.
    """

    __tablename

In [21]:
chat("Now you are going to write the endpoints using FastAPI. Write the contents of `routes/books.py` and `routes/users.py")

Here are the contents for the files `routes/books.py` and `routes/users.py`, using FastAPI:

**routes/books.py**
```python
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy import select
from models.book import Book, BookReview
from database import get_db

router = APIRouter()

@router.get("/books/", response_model=list[Book])
async def read_books(db: Session = Depends(get_db)):
    return db.execute(select(Book)).scalars().all()

@router.get("/books/{book_id}", response_model=Book)
async def read_book(book_id: int, db: Session = Depends(get_db)):
    book = db.get(Book, book_id)
    if not book:
        raise HTTPException(status_code=404, detail="Book not found")
    return book

@router.post("/books/", response_model=Book)
async def create_book(book: BookCreate, db: Session = Depends(get_db)):
    new_book = Book(**book.dict())
    db.add(new_book)
    db.commit()
    return new_book

@router.put("/books/{book_id}", response_model=Book)
async def update_book(book

In [22]:
chat("Write the contents of `main.py`")

Here are the contents for `main.py`:

```python
from fastapi import FastAPI
from routes import users, books

app = FastAPI()

app.include_router(users.router, prefix="/users")
app.include_router(books.router, prefix="/books")

@app.get("/")
async def root():
    return {"message": "Welcome to the API!"}
```

This `main.py` file sets up a FastAPI application and includes the routers for users and books. The `root` function is used as the default route for the API.

Note that you'll need to have a `routes.py` file with the following contents:

```python
from fastapi import APIRouter

users = APIRouter()
books = APIRouter()
```

This sets up two separate routers, one for users and one for books. The actual routes are defined in the `users.py` and `books.py` files.

Also, make sure to have a `uvicorn` command in your `requirements.txt` file:

```bash
uvicorn main:app --host 0.0.0.0 --port 8000 --reload
```

This will start the FastAPI application with hot reloading enabled.

In [23]:
chat("Write a bash script called `start.sh` that runs uvicorn in production mode")

Here is a simple `start.sh` script that runs uvicorn in production mode:

```bash
#!/bin/bash

# Set environment variables for production mode
export DATABASE_URL="your_database_url"
export SECRET_KEY="your_secret_key"

# Run uvicorn with production settings
uvicorn main:app --host 0.0.0.0 --port 8000 --reload --workers 4 --log-level warning
```

Replace `your_database_url` and `your_secret_key` with your actual database URL and secret key.

Note that this script assumes you have uvicorn installed in your environment. If not, you can install it using pip:

```bash
pip install uvicorn
```

Also, make sure to give execute permissions to the script by running:

```bash
chmod +x start.sh
```

You can then run the script by executing:

```bash
./start.sh
```

This will start your FastAPI application with uvicorn in production mode.

In [24]:
chat("Now generate the contents of a dockerfile that builds and runs this application, to be deployed in Azure")

Here is a Dockerfile that builds and runs your FastAPI application:

```dockerfile
# Use an official Python runtime as a parent image
FROM python:3.9-slim-buster

# Set the working directory to /app
WORKDIR /app

# Copy the requirements file into the container
COPY requirements.txt .

# Install the dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code into the container
COPY . .

# Expose the port that the application will use to communicate with the outside world
EXPOSE 8000

# Run the command to start the application when the container is launched
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"]
```

This Dockerfile does the following:

1. Uses an official Python 3.9 runtime as a parent image.
2. Sets the working directory to `/app`.
3. Copies the `requirements.txt` file into the container.
4. Installs the dependencies specified in `requirements.txt`.
5. Copies the application code into